# SETTINGS

In [105]:
############ LIBRARIES

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')
%matplotlib inline

import os
import time
import random
import multiprocessing
import pickle

import scipy.stats

import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')

import lightgbm as lgb

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.decomposition import PCA, FastICA, FactorAnalysis
from sklearn.metrics import log_loss, roc_auc_score, confusion_matrix
from sklearn.neighbors import NearestNeighbors

In [106]:
############ HELPER FUNCTIONS

import functions
import importlib
importlib.reload(functions)
from functions import *

In [107]:
############ RANDOMNESS

# seed function
def seed_everything(seed = 23):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    
# set seed
seed = 23
seed_everything(seed)

# DATA PREPARATION

In [108]:
# import CSV
df = pd.read_pickle('../input/data_v1.pkl')
print(df.shape)

(1097231, 447)


In [92]:
# target variable
target = 'isFraud'

In [93]:
# partitioning
train = df[df[target].isnull() == False]
test  = df[df[target].isnull() == True]
print(train.shape)
print(test.shape)
del df

(590540, 447)
(506691, 447)


In [94]:
# sort data
train = train.sort_values('TransactionID')
test  = test.sort_values('TransactionID')

In [95]:
# target variable
y = train[target]
g = train['DT_M']
p = train['ProductCD']
del train, test

In [96]:
### IMPORT OOF PREDS

# which model to stack?
#model = 'lgb'

# threshold
#min_auc = 90

# list names
names = sorted(os.listdir('../oof_preds'))
#names = [n for n in names if int(n[n.rindex('_')+1:-7]) > min_auc]
#names = [s for s in names if model in s]

# preprocessing loop
for name in names:

    # load preds
    tmp_tr = pd.read_csv('../oof_preds/'   + str(name))
    tmp_te = pd.read_csv('../submissions/' + str(name))

    # sort preds by ID
    tmp_tr = tmp_tr.sort_values('TransactionID')
    tmp_te = tmp_te.sort_values('TransactionID')

    # cbind data
    if name == names[0]:  
        
        tmp_tr.columns = ['TransactionID', name]    
        tmp_te.columns = ['TransactionID', name]    
        train = tmp_tr 
        test  = tmp_te
        
    else:
        
        del tmp_tr['TransactionID'], tmp_te['TransactionID']
        tmp_tr.columns = [name]    
        tmp_te.columns = [name]    
        train = pd.concat([train, tmp_tr], axis = 1)
        test  = pd.concat([test,  tmp_te], axis = 1)
        
# display information
print('- Train shape:', train.shape)
print('- Test shape:',  test.shape)

- Train shape: (590540, 49)
- Test shape: (506691, 49)


# MODELING

In [97]:
# drop bad features
excluded_feats = ['TransactionID', 'DT_M']
features = [f for f in train.columns if f not in excluded_feats]
print(train[features].shape)

(590540, 48)


In [98]:
### PARAMETERS

# settings
cores = 24
seed  = 999

# cross-validation
num_folds = 6
shuffle   = True

# muner of rounds
max_rounds = 500
stopping   = 100
verbose    = 100

# LGB parameters
lgb_params = {
    'boosting_type':     'gbdt',
    'objective':         'binary',
    'metric':            'auc',
    'bagging_fraction':  0.9,
    'feature_fraction':  0.9,
    'lambda_l1':         0.1,
    'lambda_l2':         0.1,
    'min_split_gain':    0,
    'min_child_weight':  0.1,
    'min_child_samples': 20,
    'silent':            True,
    'verbosity':         -1,
    'learning_rate':     0.1,
    'max_depth':         5,
    'num_leaves':        50,
    'scale_pos_weight':  1,
    'n_estimators':      max_rounds,
    'nthread' :          cores,
    'random_state':      seed,
}

# data partitinoing
#folds = StratifiedKFold(n_splits = num_folds, random_state = seed, shuffle = shuffle)
folds = GroupKFold(n_splits = num_folds)

In [99]:
# placeholders
clfs = []
preds_test   = np.zeros(test.shape[0])
preds_oof    = np.zeros(train.shape[0])
importances  = pd.DataFrame()

In [100]:
### CROSS-VALIDATION LOOP
cv_start  = time.time()
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train, y, groups = g)):
    
    # data partitioning
    trn_x, trn_y = train[features].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = train[features].iloc[val_idx], y.iloc[val_idx]
           
    # train lightGBM
    clf = lgb.LGBMClassifier(**lgb_params) 
    clf = clf.fit(trn_x, trn_y, 
                  eval_set              = [(trn_x, trn_y), (val_x, val_y)], 
                  eval_metric           = "auc", 
                  early_stopping_rounds = stopping,
                  verbose               = verbose)
    clfs.append(clf)
    
    # predict validation from the best iteration
    best_iter = clf.best_iteration_
       
    # predictions
    preds_oof[val_idx]    = clf.predict_proba(val_x, num_iteration = best_iter)[:, 1]
    preds_test           += clf.predict_proba(test[features], num_iteration = best_iter)[:, 1] / folds.n_splits 

    ## importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df['Feature'] = features
    fold_importance_df['Importance'] = clf.feature_importances_
    fold_importance_df['Fold'] = n_fold + 1
    importances = pd.concat([importances, fold_importance_df], axis = 0)
    
    # print performance
    print('----------------------')
    print('FOLD%2d: AUC = %.6f' % (n_fold + 1, roc_auc_score(y[val_idx], preds_oof[val_idx])))
    print('----------------------')
    print('')
        
    # clear memory
    del trn_x, trn_y, val_x, val_y
    gc.collect()
    
    
# print overall performance    
cv_perf = roc_auc_score(y, preds_oof)
print('--------------------------------')
print('- OOF AUC = %.6f' % cv_perf)
print('- CV TIME = {:.2f} min'.format((time.time() - cv_start) / 60))
print('--------------------------------')

Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.964501	valid_1's auc: 0.924974
Early stopping, best iteration is:
[24]	training's auc: 0.957295	valid_1's auc: 0.926007
----------------------
FOLD 1: AUC = 0.926007
----------------------

Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.960649	valid_1's auc: 0.95053
Early stopping, best iteration is:
[40]	training's auc: 0.955799	valid_1's auc: 0.950669
----------------------
FOLD 2: AUC = 0.950669
----------------------

Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.961285	valid_1's auc: 0.948195
Early stopping, best iteration is:
[39]	training's auc: 0.955596	valid_1's auc: 0.948698
----------------------
FOLD 3: AUC = 0.948698
----------------------

Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.961706	valid_1's auc: 0.944231
Early stopping, best iteration is:
[6]	training's au

In [101]:
############ RECHECK PERFORMANCE  

# check performance
print(np.round(roc_auc_score(y, preds_oof), 5))

0.94019


# SUBMISSION

In [102]:
# file name
model = 'stack_lgb'
perf  = str(round(cv_perf, 6))[2:7]
name  = model + str(len(features)) + '_' + perf
name

'stack_lgb48_94019'

In [103]:
# export submission
sub = pd.DataFrame({'TransactionID': test['TransactionID'], 'isFraud': preds_test})
sub.to_csv('../submissions/' + str(name) + '.csv', index = False)
sub.head()

,TransactionID,isFraud
0,3663549,0.005132
1,3663550,0.006246
2,3663551,0.005632
3,3663552,0.005281
4,3663553,0.005735


In [104]:
# check rank correlation with the best submission
from scipy.stats import spearmanr
sub  = sub.sort_values('TransactionID')
best = pd.read_csv("../submissions/BlendSolution.csv")
best = best.sort_values('TransactionID')
spearmanr(sub.isFraud, best.isFraud)

SpearmanrResult(correlation=0.8062620173867048, pvalue=0.0)